In [1]:
import numpy as np

In [2]:
class SVM:
    def __init__(self, max_iter=100, kernel='linear'):
        # 松弛变量
        self.C = 1.0
        self.E = None
        self.alpha = None
        self.b = 0.0
        # label
        self.Y = None
        # 样本
        self.X = None
        # 特征数量
        self.n = None
        # 样本数量
        self.m = None
        # 最大迭代次数
        self.max_iter = max_iter
        self._kernel = kernel

    def __init_args(self, features, labels):
        self.m, self.n = features.shape
        self.X = features
        self.Y = labels
        # 将Ei保存在一个列表里
        self.alpha = np.ones(self.m)
        self.E = [self.__E(i) for i in range(self.m)]

    def __KKT(self, i):
        """
        检查是否满足KKT条件
        :param i:
        :return:
        """
        y_g = self.__g(self.X[i]) * self.Y[i]
        if self.alpha[i] == 0:
            return y_g >= 1
        elif 0 < self.alpha[i] < self.C:
            return y_g == 1
        else:
            return y_g <= 1

    def __g(self, x):
        """
        :param i:样本index
        :return:预测值
        """
        r = self.b
        for i in range(self.m):
            r += self.alpha[i] * self.Y[i] * self.kernel(x, self.X[i])
        return r

    def kernel(self, x1, x2):
        """
        核函数
        :param x1:
        :param x2:
        :return:
        """
        if self._kernel == 'linear':
            return sum([x1[k] * x2[k] for k in range(self.n)])
        elif self._kernel == 'poly':
            return (sum([x1[k] * x2[k] for k in range(self.n)]) + 1) ** 2
        return 0

    # E（x）为g(x)对输入x的预测值和y的差
    def __E(self, i):
        return self.__g(self.X[i]) - self.Y[i]

    def _init_alpha(self):
        # 外层循环首先遍历所有满足0<a<C的样本点，检验是否满足KKT
        index_list = [i for i in range(self.m) if 0 < self.alpha[i] < self.C]
        # 否则遍历整个训练集
        non_satisfy_list = [i for i in range(self.m) if i not in index_list]
        index_list.extend(non_satisfy_list)

        for i in index_list:
            if self.__KKT(i):
                continue
            E1 = self.E[i]
            # 如果E2是+，选择最小的；如果E2是负的，选择最大的
            if E1 >= 0:
                j = min(range(self.m), key=lambda x: self.E[x])
            else:
                j = max(range(self.m), key=lambda x: self.E[x])
            return i, j

    def __compare(self, _alpha, L, H):
        """
        给新的alpha加上约束
        :param _alpha:
        :param L:
        :param H:
        :return:
        """
        if _alpha > H:
            return H
        elif _alpha < L:
            return L
        else:
            return _alpha

    def fit(self, features, labels):
        self.__init_args(features, labels)
        # 开始训练
        for t in range(self.max_iter):
            i1, i2 = self._init_alpha()
            # 边界
            if self.Y[i1] == self.Y[i2]:
                L = max(0, self.alpha[i1] + self.alpha[i2] - self.C)
                H = min(self.C, self.alpha[i1] + self.alpha[i2])
            else:
                L = max(0, self.alpha[i2] - self.alpha[i1])
                H = min(self.C, self.C + self.alpha[i2] - self.alpha[i1])

            E1 = self.E[i1]
            E2 = self.E[i2]
            # eta=K11+K22-2K12
            eta = self.kernel(self.X[i1], self.X[i1]) + self.kernel(
                self.X[i2],
                self.X[i2]) - 2 * self.kernel(self.X[i1], self.X[i2])
            if eta <= 0:
                # print('eta <= 0')
                continue

            alpha2_new_unc = self.alpha[i2] + self.Y[i2] * (
                    E1 - E2) / eta  #此处有修改，根据书上应该是E1 - E2，书上130-131页
            alpha2_new = self.__compare(alpha2_new_unc, L, H)

            alpha1_new = self.alpha[i1] + self.Y[i1] * self.Y[i2] * (
                    self.alpha[i2] - alpha2_new)

            b1_new = -E1 - self.Y[i1] * self.kernel(self.X[i1], self.X[i1]) * (
                    alpha1_new - self.alpha[i1]) - self.Y[i2] * self.kernel(
                self.X[i2],
                self.X[i1]) * (alpha2_new - self.alpha[i2]) + self.b
            b2_new = -E2 - self.Y[i1] * self.kernel(self.X[i1], self.X[i2]) * (
                    alpha1_new - self.alpha[i1]) - self.Y[i2] * self.kernel(
                self.X[i2],
                self.X[i2]) * (alpha2_new - self.alpha[i2]) + self.b

            if 0 < alpha1_new < self.C:
                b_new = b1_new
            elif 0 < alpha2_new < self.C:
                b_new = b2_new
            else:
                # 选择中点
                b_new = (b1_new + b2_new) / 2

            # 更新参数
            self.alpha[i1] = alpha1_new
            self.alpha[i2] = alpha2_new
            self.b = b_new

            self.E[i1] = self.__E(i1)
            self.E[i2] = self.__E(i2)

    def predict(self, data):
        r = self.b
        for i in range(self.m):
            r += self.alpha[i] * self.Y[i] * self.kernel(data, self.X[i])
        return 1 if r > 0 else -1

    def score(self, _X_test, _y_test):
        right_count = 0
        for i in range(len(X_test)):
            result = self.predict(_X_test[i])
            if result == y_test[i]:
                right_count += 1
        return right_count / len(_y_test)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X[:100], y[:100], test_size=0.3, random_state=3)
svm = SVM(max_iter=200)
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.5666666666666667